In [2]:
import sys
from ultralytics import YOLO
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel, QFileDialog
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
from ultralytics import YOLO
import supervision as sv
import cv2

class YOLOv8App(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.model = YOLO('seg.pt')

    def initUI(self):
        self.setWindowTitle('Bimbot Vision')
        self.setGeometry(100, 100, 800, 600)
        
        layout = QVBoxLayout()
        
        self.count = 'Seleccione una imagen para comenzar'

        self.count_label = QLabel(self.count, self)
        self.count_label.setMaximumHeight(50)
        self.count_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.count_label)

        self.label = QLabel(self)
        self.label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.label)

        self.btn = QPushButton('Cargar Imagen', self)
        self.btn.clicked.connect(self.loadImage)
        layout.addWidget(self.btn)

        self.setLayout(layout)

        with open('../Utils/style.css', 'r') as f:
            self.setStyleSheet(f.read())

    def loadImage(self):
        options = QFileDialog.Options()
        fileName, _ = QFileDialog.getOpenFileName(self, "Seleccionar Imagen", "", "Images (*.png *.xpm *.jpg)", options=options)
        if fileName:
            self.predictImage(fileName)

    def predictImage(self, filePath):
        img = cv2.imread(filePath)
        results = self.model.predict(source=img)
        print(results[0].boxes)
        n = len(results[0].boxes)
        annotated_img = results[0].plot(show=False)
        annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
        height, width, _ = annotated_img.shape
        bytesPerLine = 3 * width
        qImg = QPixmap(QImage(annotated_img.data, width, height, bytesPerLine, QImage.Format_RGB888))
        self.label.setPixmap(qImg)
        self.count_label.setText(f'Se detectan {n} cajas')

def run_app():
    app = QApplication(sys.argv)
    ex = YOLOv8App()
    ex.show()
    app.exec_()

if __name__ == '__main__':
    run_app()